In [157]:

import io
import os
%matplotlib inline
import matplotlib.pyplot as plt

# data management
import xarray as xr
import numpy as np
import pandas as pd

# Azure Connectivity
import getpass
import azure.storage.blob
from azure.storage.blob import BlobClient

# utilities
import time
import urllib.parse
from pathlib import Path
from tempfile import NamedTemporaryFile


In [166]:
sas_token = getpass.getpass() 
sas_token

'sp=racw&st=2021-09-27T07:58:58Z&se=2021-09-27T15:58:58Z&spr=https&sv=2020-08-04&sr=c&sig=GpRHin%2BVS%2FI8q3K5tNI3j0U%2F1XkJC0U1RhTGZ54GfD4%3D'

In [154]:

def azure2pc(sas_url, path):
    if Path(path).exists():
        raise FileExistsError(path)
    else: 
        blob_client = BlobClient.from_blob_url(sas_url)
        download_stream = blob_client.download_blob()
        
        with NamedTemporaryFile(dir=os.path.dirname(path)) as f:
            f.write(download_stream.readall())
            os.link(f.name, path)


def pc2azure(path, sas_url):
    blob_client = BlobClient.from_blob_url(sas_url)
    with open(path, "rb") as f:
        blob_client.upload_blob(f)


In [155]:
# Download a file from Azure to Planatery Computer VM:

# filename = "Avg_temp_max_CMIP5__30_yrs__1950_to_1979.nc"
filename = "tasmax_day_BCSD_historical_r1i1p1_inmcm4_1976.nc"

# From Azure
url_prefix = 'https://nexdcp30.blob.core.windows.net'
blob_folder = "cmip5/historical"
sas_url = f"{url_prefix}/{blob_folder}/{filename}?{sas_token}"

# To Pla. Compute remote VM
path = f"./Koray/CMIP5/{filename}"

azure2pc(sas_url, path)

In [167]:
# Upload a file from Planatery Computer VM to Azure:

filename = "test.png"

url_prefix = 'https://nexdcp30.blob.core.windows.net'
blob_folder = "flagged"
sas_url = f"{url_prefix}/{blob_folder}/{filename}?{sas_token}"

path = "Koray/test.png"

pc2azure(path, sas_url)
